In [ ]:
import torch
import torch.nn as nn

from pyfiles import *

import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   
os.environ["CUDA_VISIBLE_DEVICES"]="0"


stype = 'os'

sparsity = 0.80
float_model_path = f"result/__VGG16_{stype}_prune/model_best.pth.tar"
lr = 3e-3
epochs = 100

model_name = f"__VGG16_{stype}_prune_{sparsity:.2f}_q"
fdir = 'result/' + model_name

model = VGG16()
attach_prune(model)
checkpoint = torch.load(float_model_path, weights_only=False)
model.load_state_dict(checkpoint['state_dict'])
model.cuda()

criterion = nn.CrossEntropyLoss().cuda()
optimizer = torch.optim.AdamW(model.parameters(), lr=lr)

In [ ]:
val_model(model)

In [ ]:
quantize_pruned(model)

val_model(model)

In [ ]:
train_model(model, fdir, criterion, optimizer, epochs, stype=stype)

In [ ]:
model = VGG16().cuda()
attach_prune(model)
quantize_pruned(model)

checkpoint = torch.load(f'{fdir}/model_best.pth.tar', weights_only=False)
model.load_state_dict(checkpoint['state_dict'])
model.cuda()

l_to_save = 27

class SaveOutput:
    def __init__(self):
        self.outputs = []
    def __call__(self, module, module_in):
        self.outputs.append(module_in)
    def clear(self):
        self.outputs = []
save_output = SaveOutput()
model.features[l_to_save].register_forward_pre_hook(save_output)

print(f'Acc: {val_model(model):.2%}')

print_sparsity(model)

In [ ]:
l = model.features[l_to_save]
nz_idx = (l.weight_q.sum(dim=-1).sum(dim=-1) != 0).nonzero()
print(l.weight_q[nz_idx[:,0],nz_idx[:,1],:,:][-4:])

In [ ]:
print(save_output.outputs[0][0][:2,:2])

In [ ]:
print(f'Weight int: \n{(model.features[l_to_save].weight_q.data / (model.features[l_to_save].weight_quant.wgt_alpha.data.item()/(2**(4-1)-1)))[0,nz_idx[0]]}')
x = save_output.outputs[0][0]
x_alpha = model.features[l_to_save].act_alpha.data.item()
x_delta = x_alpha / (2**(4)-1)
act_q = act_quantization(4)
x_q = act_q(x, x_alpha)
print(f'Act int: \n{(x_q/x_delta)[:2,:2]}')